In [7]:
from yfinance import *
import pandas as pd
import sqlite3

# Tickers for our stocks
txn = Ticker("TXN")
regn = Ticker("REGN")
amd = Ticker("AMD")
gral = Ticker("GRAL")
klac = Ticker("KLAC")
market = Ticker("^GSPC")


# Historical data YTD
txn_ytd = txn.history(period="ytd")
regn_ytd = regn.history(period="ytd")
amd_ytd = amd.history(period="ytd")
gral_ytd = gral.history(period="ytd")
klac_ytd = klac.history(period="ytd")
market_ytd = market.history(period="ytd")


# Trend function, returns list of floats of % up/down each day
def get_trend(ytd_data):
    trend = ((ytd_data['Close'] - ytd_data['Open']) / ytd_data['Open']) * 100
    return trend


# Trends for all stocks along with ^GSPC
market_trend = get_trend(market_ytd)
txn_trend = get_trend(txn_ytd)
regn_trend = get_trend(regn_ytd)
amd_trend = get_trend(amd_ytd)
gral_trend = get_trend(gral_ytd)
klac_trend = get_trend(klac_ytd)


# These loops mark True for a positive shift (Stock went up) and False for a negative shift (Market went down)
market_arr = []
for tr in market_trend:
    tr = tr > 0
    market_arr.append(tr)


txn_arr = []
for tr in txn_trend:
    tr = tr > 0
    txn_arr.append(tr)



# Data With %Yield
market_data = {"Date": market_ytd.index, "Trend": market_trend}

txn_data = {"Date": txn_ytd.index, "Trend": txn_trend}
regn_data = {"Date": regn_ytd.index, "Trend": regn_trend}
amd_data = {"Date": amd_ytd.index, "Trend": amd_trend}
gral_data = {"Date": gral_ytd.index, "Trend": gral_trend}
klac_data = {"Date": klac_ytd.index, "Trend": klac_trend}


# With Boolean markers
# market_data = {"Date": market_ytd.index, "Trend": market_arr}

# txn_data = {"Date": txn_ytd.index, "Trend": txn_arr}

# Dataframes for our stocks and ^GSPC
txn_df = pd.DataFrame(txn_data)
regn_df = pd.DataFrame(regn_data)
amd_df = pd.DataFrame(amd_data)
gral_df = pd.DataFrame(gral_data)
klac_df = pd.DataFrame(klac_data)

market_df = pd.DataFrame(market_data)

# Local Memory (SQLITE)
conn = sqlite3.connect(':memory:')

# Importing Dataframes into SQL
txn_df.to_sql('txn_data', conn, index=False)
regn_df.to_sql('regn_data', conn, index=False)
amd_df.to_sql('amd_data', conn, index=False)
gral_df.to_sql('gral_data', conn, index=False)
klac_df.to_sql('klac_data', conn, index=False)

market_df.to_sql('market_data', conn, index=False)

# Querying function to find dates when TXN (or any other stock) went up when the Market was down
def get_trend_data(data):
    query = f"""
    SELECT Date, Trend
    FROM {data}
    WHERE (Trend > 0
        AND Date IN(
            SELECT Date
            FROM market_data
            WHERE Trend < 0 ));
    """
    return query


# Result
# result = pd.read_sql_query(get_trend_data(txn_data), conn)

# Output function
def show_all(ticker, data):
    result = pd.read_sql_query(get_trend_data(data), conn)
    print(f"\n{ticker} Outperformance Dates & Trend vs. ^GSPC")
    print(result)

show_all("TXN", 'txn_data')
show_all("REGN", 'regn_data')
show_all("AMD", 'amd_data')
show_all("GRAL", 'gral_data')
show_all("KLAC", 'klac_data')


# CSV Files
txn_df.to_csv('txn_trend.csv', index=False)
regn_df.to_csv('regn_trend.csv', index=False)
amd_df.to_csv('amd_trend.csv', index=False)
gral_df.to_csv('gral_trend.csv', index=False)
klac_df.to_csv('klac_trend.csv', index=False)
market_df.to_csv('market_trend.csv', index=False)







TXN Outperformance Dates & Trend vs. ^GSPC
                         Date     Trend
0   2025-01-06 00:00:00-05:00  1.014451
1   2025-01-14 00:00:00-05:00  1.023288
2   2025-01-31 00:00:00-05:00  0.451628
3   2025-02-14 00:00:00-05:00  0.837422
4   2025-02-20 00:00:00-05:00  2.327915
5   2025-03-26 00:00:00-04:00  0.124830
6   2025-04-17 00:00:00-04:00  0.760246
7   2025-04-21 00:00:00-04:00  0.103117
8   2025-05-09 00:00:00-04:00  1.293583
9   2025-05-20 00:00:00-04:00  0.196459
10  2025-05-28 00:00:00-04:00  0.436317
11  2025-06-04 00:00:00-04:00  0.958127

REGN Outperformance Dates & Trend vs. ^GSPC
                         Date     Trend
0   2025-01-06 00:00:00-05:00  0.176225
1   2025-01-07 00:00:00-05:00  1.948790
2   2025-01-29 00:00:00-05:00  0.190973
3   2025-02-20 00:00:00-05:00  1.158351
4   2025-02-21 00:00:00-05:00  0.343878
5   2025-02-24 00:00:00-05:00  1.323996
6   2025-02-25 00:00:00-05:00  0.481940
7   2025-03-06 00:00:00-05:00  2.245649
8   2025-03-10 00:00:00-04:00  